<a href="https://colab.research.google.com/github/melindasiburian/melindacode/blob/main/Tugas_time_series_DM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Nama: Melinda Venta Lydia Siburian**


**NIM : 23031554124**


**Kelas : 2023A**

In [1]:
import datetime as dt
import pandas as pd
from pandas_datareader import data as pdr
import yfinance as yf
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

# DATA EXPLORATION

Mengambil data historis saham dari tahun 2000 hingga sekarang

In [2]:
end=dt.datetime.now()
start=dt.datetime(2000,1,1)
start,end

(datetime.datetime(2000, 1, 1, 0, 0),
 datetime.datetime(2025, 3, 27, 14, 44, 50, 861035))

**SELECT THE STOCK TO ANALYZE**

membuat daftar simbol saham yang sesuai dengan format Yahoo Finance agar bisa digunakan untuk mengambil data saham dari Bursa Efek Australia (ASX).


*   CBA = Commonwealth Bank of Australia
*   NAB = National Australia Bank







In [3]:
stocklist = ['CBA','NAB']
stocks = [i +'.AX' for i in stocklist]
stocks

['CBA.AX', 'NAB.AX']

mengunduh data historis saham dari Yahoo Finance menggunakan library yfinance.



In [4]:
# Mengunduh data langsung dengan yfinance
df = yf.download(stocks, start=start, end=end)

# Menampilkan 5 baris pertama
print(df.head())

YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  2 of 2 completed

Price          Close                High                 Low            \
Ticker        CBA.AX    NAB.AX    CBA.AX    NAB.AX    CBA.AX    NAB.AX   
Date                                                                     
2000-01-03  7.119414  5.492553  7.119414  5.492553  7.119414  5.492553   
2000-01-04  7.037988  5.303967  7.111272  5.421833  7.018988  5.280394   
2000-01-05  6.880561  5.197887  6.929418  5.228533  6.853419  5.150741   
2000-01-06  6.744851  5.115383  6.921276  5.256822  6.731280  5.115383   
2000-01-07  6.812708  5.160172  6.853421  5.174316  6.750280  5.110668   

Price           Open             Volume           
Ticker        CBA.AX    NAB.AX   CBA.AX   NAB.AX  
Date                                              
2000-01-03  7.119414  5.492553        0        0  
2000-01-04  7.111272  5.419476  1018072  3005390  
2000-01-05  6.918832  5.183743  2191283  4604126  
2000-01-06  6.921276  5.255407  1505531  2574906  
2000-01-07  6.798594  5.138956  1414040  3028993  

df.index mengembalikan DatetimeIndex, yaitu deretan tanggal dalam periode yang telah ditentukan (start sampai end).

In [5]:
df.index

DatetimeIndex(['2000-01-03', '2000-01-04', '2000-01-05', '2000-01-06',
               '2000-01-07', '2000-01-10', '2000-01-11', '2000-01-12',
               '2000-01-13', '2000-01-14',
               ...
               '2025-03-14', '2025-03-17', '2025-03-18', '2025-03-19',
               '2025-03-20', '2025-03-21', '2025-03-24', '2025-03-25',
               '2025-03-26', '2025-03-27'],
              dtype='datetime64[ns]', name='Date', length=6422, freq=None)

Kolom pada data saham


*   Open → Harga pembukaan saham
*   High → Harga tertinggi dalam satu hari
*   Low → Harga terendah dalam satu hari
*   Close → Harga penutupan saham
*   Volume → Jumlah saham yang diperdagangkan
















In [6]:
df.columns

MultiIndex([( 'Close', 'CBA.AX'),
            ( 'Close', 'NAB.AX'),
            (  'High', 'CBA.AX'),
            (  'High', 'NAB.AX'),
            (   'Low', 'CBA.AX'),
            (   'Low', 'NAB.AX'),
            (  'Open', 'CBA.AX'),
            (  'Open', 'NAB.AX'),
            ('Volume', 'CBA.AX'),
            ('Volume', 'NAB.AX')],
           names=['Price', 'Ticker'])

mengambil kolom "Close" (harga penutupan saham) dari DataFrame df dan menampilkan 5 baris pertama.

In [7]:
Close=df.Close
Close.head()

Ticker,CBA.AX,NAB.AX
Date,,
2000-01-03,7.119414,5.492553
2000-01-04,7.037988,5.303967
2000-01-05,6.880561,5.197887
2000-01-06,6.744851,5.115383
2000-01-07,6.812708,5.160172


Menganalisis statistik harga penutupan saham dalam 100 hari terakhir.
Interpretasi Output


1.   Rata-rata (mean) harga penutupan dalam 100 hari terakhir saham CBA.AX adalah 753.784081 dan rata-rata harga penutupan dalam 100 hari terakhir saham NAB.AX adalah 37.030000.
2.   Standar deviasi (std) saham CBA.AX adalah 5.267555, menunjukkan fluktuasi sekitar ±5.27 poin dari rata-rata. Sedangkan standar deviasi (std) saham NAB.AX adalah 2.501833, menunjukkan fluktuasi sekitar ±2.50 poin dari rata-rata.
3. Harga minimum (min) saham CBA.AX adalah 142.360001 dan harga minimum (min) saham NAB.AX adalah 32.590000. Harga maksimum (max) saham CBA.AX adalah 164.412994 dan Harga maksimum (max) saham NAB.AX adalah 41.240002
4. 10% persentil saham CBA.AX sebesar 145.797998 berarti dalam 10% hari dalam periode analisis, harga saham CBA.AX berada di bawah 145.80 sedangkan 10% persentil saham NAB.AX sebesar 33.348000 berarti dalam 10% hari dalam periode analisis, harga saham CBA.AX berada di bawah 33.35
5. 50% persentil saham CBA.AX sebesar 154.048431 menunjukkan bahwa 50% dari seluruh hari dalam periode analisis memiliki harga penutupan di bawah 154.05. Sedangkan 50% persentil saham NAB.AX sebesar 37.389999 menunjukkan bahwa 50% dari seluruh hari dalam periode analisis memiliki harga penutupan di bawah 37.39
6. 90% persentil saham CBA.AX sebesar 160.365784  berarti 90% dari seluruh hari dalam periode analisis memiliki harga penutupan di bawah 160.37. Sedangkan 90% persentil saham NAB.AX sebesar 40.400001 berarti 90% dari seluruh hari dalam periode analisis memiliki harga penutupan di bawah 40.40


In [8]:
Close[Close.index > end-dt.timedelta(days=100)].describe(percentiles=[0.1,0.5,0.9])

Ticker,CBA.AX,NAB.AX
count,68.000000,68.000000
mean,153.729021,36.986912
std,5.247776,2.508385
min,142.360001,32.590000
10%,145.830997,33.356001
50%,154.043495,37.315001
90%,160.361839,40.395001
max,164.412994,41.240002


Perbandingan ini menunjukkan bahwa CBA.AX memiliki volatilitas lebih tinggi dibandingkan NAB.AX, dengan pergerakan harga hampir dua kali lebih besar. Hal ini mengindikasikan bahwa CBA.AX lebih berisiko tetapi juga memiliki potensi keuntungan lebih besar. Sebaliknya, NAB.AX lebih stabil, dengan pergerakan harga yang lebih kecil dan lebih terkendali, sehingga lebih cocok bagi investor yang mengutamakan kestabilan dan risiko lebih rendah.

In [9]:
import plotly.offline as pyo
pyo.init_notebook_mode(connected=True)

pd.options.plotting.backend= 'plotly'

In [10]:
import plotly.io as pio
pio.renderers.default = 'colab'


In [11]:
# Plot interaktif
fig = df['Close'].plot()
fig.show()

persentase perubahan harian pada harga penutupan saham CBA.AX menggunakan metode .pct_change()

In [12]:
df_returns1 = Close['CBA.AX'].pct_change()
print(df_returns1.head())

Date
2000-01-03         NaN
2000-01-04   -0.011437
2000-01-05   -0.022368
2000-01-06   -0.019724
2000-01-07    0.010061
Name: CBA.AX, dtype: float64


persentase perubahan harian pada harga penutupan saham NAB.AX menggunakan metode .pct_change()

In [13]:
df_returns2 = Close['NAB.AX'].pct_change()
print(df_returns2.head())

Date
2000-01-03         NaN
2000-01-04   -0.034335
2000-01-05   -0.020000
2000-01-06   -0.015873
2000-01-07    0.008756
Name: NAB.AX, dtype: float64


Histogram menunjukkan distribusi perubahan harga yang lebih terpusat di sekitar nol. Sebagian besar perubahan harian berada dalam kisaran kecil, tetapi ada beberapa ekor panjang yang menunjukkan perubahan besar dalam harga saham.

In [14]:
return1=Close['CBA.AX'].pct_change().plot(kind='hist')
return1

Distribusi perubahan harga juga terpusat di sekitar nol, tetapi tampaknya sedikit lebih lebar dibandingkan dengan CBA.AX. Ada lebih banyak penyebaran di ekor distribusi, yang mungkin menunjukkan volatilitas yang sedikit lebih tinggi dalam beberapa kasus.

In [15]:
return2=Close['NAB.AX'].pct_change().plot(kind='hist')
return2

Secara rata-rata, saham CBA.AX memiliki tingkat pertumbuhan harian yang sedikit lebih tinggi dibandingkan dengan saham NAB.AX. Namun, perbedaannya cukup kecil, yang berarti kedua saham cenderung memiliki pertumbuhan jangka panjang yang sebanding.

Volatilitas harian mengukur seberapa besar fluktuasi harga saham dari hari ke hari. Dalam hal ini, NAB.AX memiliki volatilitas yang lebih tinggi dibandingkan dengan CBA.AX (1.51% vs 1.37%). Ini berarti bahwa harga saham NAB.AX mengalami perubahan yang lebih besar dan lebih sering dibandingkan dengan CBA.AX, menjadikannya sedikit lebih berisiko.

In [16]:
print("Rata-rata return harian:")
print(df_returns1.mean())
print(df_returns2.mean())

print("\nVolatilitas harian:")
print(df_returns1.std())
print(df_returns2.std())


Rata-rata return harian:
0.0005682076743802607
0.00039858023854909047

Volatilitas harian:
0.0136744769040902
0.015091444125939392


# PEMODELAN DAN PREDIKSI

Data saham dipisahkan menjadi dua data yang berbeda untuk mempermudah pemodelan

In [17]:
df_CBA = df.xs('CBA.AX', level=1, axis=1)  # Ambil semua atribut BBCA
df_NAB = df.xs('NAB.AX', level=1, axis=1)  # Ambil semua atribut TLKM

# Tampilkan hasil
print(df_CBA.head())
print(df_NAB.head())


Price          Close      High       Low      Open   Volume
Date                                                       
2000-01-03  7.119414  7.119414  7.119414  7.119414        0
2000-01-04  7.037988  7.111272  7.018988  7.111272  1018072
2000-01-05  6.880561  6.929418  6.853419  6.918832  2191283
2000-01-06  6.744851  6.921276  6.731280  6.921276  1505531
2000-01-07  6.812708  6.853421  6.750280  6.798594  1414040
Price          Close      High       Low      Open   Volume
Date                                                       
2000-01-03  5.492553  5.492553  5.492553  5.492553        0
2000-01-04  5.303967  5.421833  5.280394  5.419476  3005390
2000-01-05  5.197887  5.228533  5.150741  5.183743  4604126
2000-01-06  5.115383  5.256822  5.115383  5.255407  2574906
2000-01-07  5.160172  5.174316  5.110668  5.138956  3028993


In [18]:
df_CBA_c = df['Close']['CBA.AX'].to_frame().rename(columns={'CBA.AX': 'Close_CBA'})
df_NAB_c = df['Close']['NAB.AX'].to_frame().rename(columns={'NAB.AX': 'Close_NAB'})

print(df_CBA_c.head())  # Data harga saham BBCA
print(df_NAB_c.head())  # Data harga saham TLKM


            Close_CBA
Date                 
2000-01-03   7.119414
2000-01-04   7.037988
2000-01-05   6.880561
2000-01-06   6.744851
2000-01-07   6.812708
            Close_NAB
Date                 
2000-01-03   5.492553
2000-01-04   5.303967
2000-01-05   5.197887
2000-01-06   5.115383
2000-01-07   5.160172


menambahkan kolom baru bernama "Tomorrow", yang berisi harga penutupan (Close) pada hari berikutnya.

.shift(-1) menggeser nilai ke atas sebanyak satu baris, sehingga harga penutupan untuk hari berikutnya muncul dalam baris yang sama sebagai "Tomorrow".

In [19]:
df_CBA["Tomorrow"] = df_CBA["Close"].shift(-1)
df_CBA

<ipython-input-19-9c7528a2e8e9>:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



Price,Close,High,Low,Open,Volume,Tomorrow
Date,,,,,,
2000-01-03,7.119414,7.119414,7.119414,7.119414,0,7.037988
2000-01-04,7.037988,7.111272,7.018988,7.111272,1018072,6.880561
2000-01-05,6.880561,6.929418,6.853419,6.918832,2191283,6.744851
2000-01-06,6.744851,6.921276,6.731280,6.921276,1505531,6.812708
2000-01-07,6.812708,6.853421,6.750280,6.798594,1414040,6.975558
...,...,...,...,...,...,...
2025-03-21,145.600006,146.949997,145.199997,145.699997,6482426,147.630005
2025-03-24,147.630005,147.850006,146.289993,146.750000,1308316,148.630005
2025-03-25,148.630005,149.770004,147.899994,148.449997,1599602,150.229996


In [20]:
df_NAB["Tomorrow"] = df_NAB["Close"].shift(-1)
df_NAB

<ipython-input-20-78657fbdf404>:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



Price,Close,High,Low,Open,Volume,Tomorrow
Date,,,,,,
2000-01-03,5.492553,5.492553,5.492553,5.492553,0,5.303967
2000-01-04,5.303967,5.421833,5.280394,5.419476,3005390,5.197887
2000-01-05,5.197887,5.228533,5.150741,5.183743,4604126,5.115383
2000-01-06,5.115383,5.256822,5.115383,5.255407,2574906,5.160172
2000-01-07,5.160172,5.174316,5.110668,5.138956,3028993,5.233247
...,...,...,...,...,...,...
2025-03-21,33.180000,33.430000,32.990002,33.130001,11440658,33.910000
2025-03-24,33.910000,33.910000,33.180000,33.230000,4515383,33.750000
2025-03-25,33.750000,34.389999,33.709999,34.139999,4691013,33.980000


membuat kolom "Target" yang menunjukkan apakah harga saham naik atau turun pada hari berikutnya.

Jika harga penutupan besok lebih besar dari harga penutupan hari ini → 1 (saham naik).

Jika harga penutupan besok lebih kecil atau sama dengan harga penutupan hari ini → 0 (saham turun atau tetap).

In [21]:
df_CBA['Target'] = (df_CBA['Tomorrow'] > df_CBA['Close']).astype(int)
df_CBA

<ipython-input-21-e6c819681bff>:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



Price,Close,High,Low,Open,Volume,Tomorrow,Target
Date,,,,,,,
2000-01-03,7.119414,7.119414,7.119414,7.119414,0,7.037988,0
2000-01-04,7.037988,7.111272,7.018988,7.111272,1018072,6.880561,0
2000-01-05,6.880561,6.929418,6.853419,6.918832,2191283,6.744851,0
2000-01-06,6.744851,6.921276,6.731280,6.921276,1505531,6.812708,1
2000-01-07,6.812708,6.853421,6.750280,6.798594,1414040,6.975558,1
...,...,...,...,...,...,...,...
2025-03-21,145.600006,146.949997,145.199997,145.699997,6482426,147.630005,1
2025-03-24,147.630005,147.850006,146.289993,146.750000,1308316,148.630005,1
2025-03-25,148.630005,149.770004,147.899994,148.449997,1599602,150.229996,1


In [22]:
df_NAB['Target'] = (df_NAB['Tomorrow'] > df_NAB['Close']).astype(int)
df_NAB

<ipython-input-22-0c1b87dd4022>:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



Price,Close,High,Low,Open,Volume,Tomorrow,Target
Date,,,,,,,
2000-01-03,5.492553,5.492553,5.492553,5.492553,0,5.303967,0
2000-01-04,5.303967,5.421833,5.280394,5.419476,3005390,5.197887,0
2000-01-05,5.197887,5.228533,5.150741,5.183743,4604126,5.115383,0
2000-01-06,5.115383,5.256822,5.115383,5.255407,2574906,5.160172,1
2000-01-07,5.160172,5.174316,5.110668,5.138956,3028993,5.233247,1
...,...,...,...,...,...,...,...
2025-03-21,33.180000,33.430000,32.990002,33.130001,11440658,33.910000,1
2025-03-24,33.910000,33.910000,33.180000,33.230000,4515383,33.750000,0
2025-03-25,33.750000,34.389999,33.709999,34.139999,4691013,33.980000,1


mempersiapkan model machine learning menggunakan algoritma random forest untuk memprediksi apakah harga saham CBA.AX akan naik atau turun berdasarkan data historis. Model akan digunakan untuk membuat prediksi pada data testing atau data baru di masa depan.

Model akan menggunakan 5 fitur harga saham dan volume perdagangan untuk memprediksi pergerakan harga.

Target adalah 1 jika harga naik, 0 jika turun atau tetap.


In [23]:
from sklearn.ensemble import RandomForestClassifier

# Pisahkan data 80% training, 20% testing
train1, test1 = train_test_split(df_CBA, test_size=0.2, random_state=1, shuffle=False)

model1 = RandomForestClassifier(n_estimators=100,min_samples_split=100,random_state=1)
#train = df_CBA.iloc[:100]
#test = df_CBA.iloc[100:]

predictors_df1 = ['Close','Volume','Open','High','Low']
model1.fit(train1[predictors_df1],train1['Target'])

RandomForestClassifier(min_samples_split=100, random_state=1)

In [24]:
# Pisahkan data 80% training, 20% testing
train2, test2 = train_test_split(df_NAB, test_size=0.2, random_state=1, shuffle=False)

model2 = RandomForestClassifier(n_estimators=100,min_samples_split=100,random_state=1)
#train = df_NAB.iloc[:100]
#test = df_NAB.iloc[100:]

predictors_df2 = ['Close','Volume','Open','High','Low']
model2.fit(train2[predictors_df2],train2['Target'])

RandomForestClassifier(min_samples_split=100, random_state=1)

In [25]:
from sklearn.metrics import precision_score

preds1= model1.predict(test1[predictors_df1])

In [26]:
preds1=pd.Series(preds1, index=test1.index)
preds1

,0
Date,
2020-03-02,0
2020-03-03,0
2020-03-04,1
2020-03-05,1
2020-03-06,1
...,...
2025-03-21,0
2025-03-24,0
2025-03-25,0


In [27]:
from sklearn.metrics import precision_score

preds2= model2.predict(test2[predictors_df2])

In [28]:
preds2=pd.Series(preds2, index=test2.index)
preds2

,0
Date,
2020-03-02,0
2020-03-03,0
2020-03-04,0
2020-03-05,0
2020-03-06,0
...,...
2025-03-21,0
2025-03-24,0
2025-03-25,0


Model prediksi untuk CBA.AX memiliki precision lebih tinggi (51.58%) dibandingkan NAB.AX (48.32%).

Artinya, ketika model memprediksi harga saham CBA.AX akan naik, lebih sering prediksi tersebut benar dibandingkan model NAB.AX.

In [29]:
precision_score(test1['Target'],preds1)

0.5

In [30]:
precision_score(test2['Target'],preds2)

0.5069444444444444

1. Recall (Kemampuan Model Menangkap Kelas "1")
    *   CBA.AX → 0.07
    *   NAB.AX → 0.10

  Kesimpulan: Kedua model memiliki recall yang sangat rendah untuk kelas "1" (naik). Artinya, banyak sinyal kenaikan harga yang terlewatkan oleh model. Model lebih sering memprediksi harga akan turun (kelas "0").

2. F1-Score (Keseimbangan Precision & Recall)
    *   CBA.AX → 0.12
    *   NAB.AX → 0.16

  Kesimpulan: Model untuk NAB.AX memiliki sedikit keunggulan dalam keseimbangan precision-recall, meskipun secara precision lebih rendah.Namun, kedua model masih lemah dalam menangkap sinyal kenaikan harga.

3. Akurasi Keseluruhan
    *   CBA.AX → 0.47 (47%)
    *   NAB.AX → 0.45 (45%)
  
  Kesimpulan: Kedua model tidak jauh berbeda dalam akurasi keseluruhan, tapi tetap di bawah 50%, yang artinya performa masih kurang baik.

4. Performa untuk Kelas "0" (Penurunan Harga)
    *   CBA.AX: Precision 0.46, Recall 0.93 → Lebih sering benar saat memprediksi harga turun.
    *   NAB.AX: Precision 0.45, Recall 0.87 → Juga lebih dominan memprediksi penurunan harga.

  Kesimpulan: Kedua model lebih sering memprediksi penurunan harga. Namun, ini membuat model tidak seimbang, karena kenaikan harga sering kali gagal diprediksi.

In [31]:
report_df1 = classification_report(test1['Target'], preds1)
print(report_df1)

              precision    recall  f1-score   support

           0       0.46      0.92      0.61       597
           1       0.50      0.07      0.12       688

    accuracy                           0.46      1285
   macro avg       0.48      0.49      0.37      1285
weighted avg       0.48      0.46      0.35      1285



In [32]:
report_df2 = classification_report(test2['Target'], preds2)
print(report_df2)

              precision    recall  f1-score   support

           0       0.45      0.88      0.60       586
           1       0.51      0.10      0.17       699

    accuracy                           0.46      1285
   macro avg       0.48      0.49      0.38      1285
weighted avg       0.48      0.46      0.37      1285



In [33]:
combined1 = pd.concat([test1['Target'],preds1], axis=1)
combined1.plot()

In [34]:
combined2 = pd.concat([test2['Target'],preds2], axis=1)
combined2.plot()

melatih model, membuat prediksi, dan menggabungkan hasil prediksi dengan target asli dalam satu DataFrame.

In [35]:
def predict1 (train1,test1,predictors_df1,model1):
  model1.fit(train1[predictors_df1],train1['Target'])
  preds1 = model1.predict(test1[predictors_df1])
  preds1 = pd.Series(preds1, index=test1.index, name='Predictions')
  combined1 = pd.concat([test1['Target'],preds1], axis=1)
  return combined1

In [36]:
def predict2 (train2,test2,predictors_df2,model2):
  model2.fit(train2[predictors_df2],train2['Target'])
  preds2 = model2.predict(test2[predictors_df2])
  preds2 = pd.Series(preds2, index=test2.index, name='Predictions')
  combined2 = pd.concat([test2['Target'],preds2], axis=1)
  return combined2

melakukan backtesting pada model prediksi saham dengan cara melatih dan menguji model secara bertahap dalam periode waktu yang berbeda.

start1=2500: Mulai melakukan prediksi setelah 2500 data pertama digunakan sebagai training awal

step1=250: Model akan diperbarui setiap 250 data baru

In [37]:
def backtest1(data1, model1, predictors_df1, start1=2500, step1=250):
  all_predictions1 = []

  for i in range(start1, data1.shape[0], step1):
    train1 = data1.iloc[0:i].copy()
    test1 = data1.iloc[i:(i+step1)].copy()
    predictions1 = predict1(train1, test1, predictors_df1, model1)
    all_predictions1.append(predictions1)
  return pd.concat(all_predictions1)

In [38]:
def backtest2(data2, model2, predictors_df2, start2=2500, step2=250):
  all_predictions2 = []

  for i in range(start2, data2.shape[0], step2):
    train2 = data2.iloc[0:i].copy()
    test2 = data2.iloc[i:(i+step2)].copy()
    predictions2 = predict2(train2, test2, predictors_df2, model2)
    all_predictions2.append(predictions2)
  return pd.concat(all_predictions2)

In [39]:
predictions1 = backtest1(df_CBA, model1, predictors_df1)

In [40]:
predictions2 = backtest2(df_NAB, model2, predictors_df2)

menghitung jumlah prediksi yang dibuat oleh model dalam backtesting.

Saham CBA.AX lebih cenderung diprediksi turun oleh model dibanding NAB.AX.

NAB.AX memiliki lebih banyak prediksi naik, menunjukkan model melihat lebih banyak peluang kenaikan di saham ini.

In [41]:
predictions1["Predictions"].value_counts()

,count
Predictions,
0,2890
1,1032


In [42]:
predictions2["Predictions"].value_counts()

,count
Predictions,
0,2389
1,1533


In [43]:
precision_score(predictions1['Target'], predictions1['Predictions'])

0.5377906976744186

In [44]:
precision_score(predictions2['Target'], predictions2['Predictions'])

0.5068493150684932

1. Akurasi Model
    *   CBA.AX → 50%
    *   NAB.AX → 48%
  
  kurasi 50% menunjukkan bahwa model tidak jauh lebih baik daripada tebakan acak (coin flip). Ini mengindikasikan bahwa model mungkin tidak cukup baik dalam menangkap pola pergerakan harga saham.

2. Model untuk Saham CBA.AX
  
  Recall kelas "0" (turun) tinggi (0.75) → Model cukup baik dalam mengenali hari saat harga turun.

  Recall kelas "1" (naik) rendah (0.27) → Model sering gagal menangkap kenaikan harga (banyak false negatives).

  Precision untuk naik (0.54) lebih tinggi dibanding turun (0.48) → Saat model memprediksi harga naik, lebih dari separuhnya benar.

  Kesimpulan: Model lebih cenderung memprediksi turun, tetapi masih bisa menangkap sebagian hari naik.

3. Model untuk Saham NAB.AX
  
  Recall kelas "0" (turun) sangat tinggi (0.95) → Model hampir selalu benar dalam mendeteksi hari turun.

  Recall kelas "1" (naik) sangat rendah (0.05) → Model sangat buruk dalam mengenali hari naik, hampir selalu salah (false negatives sangat tinggi).

  Precision untuk naik (0.52) dan turun (0.48) hampir seimbang, tetapi recall buruk untuk naik.

  Kesimpulan: Model sangat bias ke arah prediksi turun, sehingga tidak bisa menangkap peluang kenaikan harga dengan baik.

In [45]:
report_CBA = classification_report(predictions1['Target'], predictions1['Predictions'])
print(report_CBA)

              precision    recall  f1-score   support

           0       0.48      0.75      0.59      1873
           1       0.54      0.27      0.36      2049

    accuracy                           0.50      3922
   macro avg       0.51      0.51      0.47      3922
weighted avg       0.51      0.50      0.47      3922



In [69]:
report_NAB = classification_report(predictions2['Target'], predictions2['Predictions'])
print(report_NAB)

              precision    recall  f1-score   support

           0       0.48      0.95      0.64      1401
           1       0.52      0.05      0.10      1520

    accuracy                           0.48      2921
   macro avg       0.50      0.50      0.37      2921
weighted avg       0.50      0.48      0.36      2921



menghitung proporsi atau distribusi kelas target dalam dataset

In [47]:
predictions1['Target'].value_counts() / predictions1.shape[0]

,count
Target,
1,0.522438
0,0.477562


In [48]:
predictions2['Target'].value_counts() / predictions2.shape[0]

,count
Target,
1,0.517593
0,0.482407


Membuat fitur baru berdasarkan pergerakan rata-rata dan tren harga saham dalam dataset

horizons1 adalah daftar periode waktu yang digunakan untuk membuat indikator teknikal. Nilai dalam daftar ini berarti jumlah hari yang digunakan untuk perhitungan rata-rata dan tren, misalnya 2 hari, 5 hari, 60 hari, 250 hari, dan 1000 hari.

In [49]:
horizons1 = [2,5,60,250,1000]
new_predictors1 = []

for horizon1 in horizons1 :
  rolling_averages1 = df_CBA.rolling(horizon1).mean()

  ratio_column1 = f"Close_Ratio_{horizon1}"
  df_CBA[ratio_column1] = df_CBA['Close'] / rolling_averages1 ['Close']

  trend_column1 = f"Trend_{horizon1}"
  df_CBA[trend_column1] = df_CBA.shift(1).rolling(horizon1).sum()["Target"]

  new_predictors1 += [ratio_column1, trend_column1]

<ipython-input-49-e1c9347775e8>:8: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-49-e1c9347775e8>:11: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [50]:
horizons2 = [2,5,60,250,1000]
new_predictors2 = []

for horizon2 in horizons2 :
  rolling_averages2 = df_NAB.rolling(horizon2).mean()

  ratio_column2 = f"Close_Ratio_{horizon2}"
  df_NAB[ratio_column2] = df_NAB['Close'] / rolling_averages2 ['Close']

  trend_column2 = f"Trend_{horizon2}"
  df_NAB[trend_column2] = df_NAB.shift(1).rolling(horizon2).sum()["Target"]

  new_predictors2 += [ratio_column2, trend_column2]

<ipython-input-50-06135f12000f>:8: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-50-06135f12000f>:11: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



Close_Ratio_{horizon1} = Rasio harga penutupan (Close) terhadap rata-rata bergeraknya.

Jika nilai rasio >1, berarti harga sekarang lebih tinggi dari rata-rata selama horizon1 hari.

Jika nilai rasio <1, berarti harga sekarang lebih rendah dari rata-rata.

Trend_{horizon1} menunjukkan jumlah hari dalam horizon1 hari terakhir di mana harga naik (Target = 1).

Nilai tinggi dalam Trend_{horizon1} berarti harga saham sering naik dalam periode tersebut.

In [51]:
df_CBA

Price,Close,High,Low,Open,Volume,Tomorrow,Target,Close_Ratio_2,Trend_2,Close_Ratio_5,Trend_5,Close_Ratio_60,Trend_60,Close_Ratio_250,Trend_250,Close_Ratio_1000,Trend_1000
Date,,,,,,,,,,,,,,,,,
2000-01-03,7.119414,7.119414,7.119414,7.119414,0,7.037988,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-01-04,7.037988,7.111272,7.018988,7.111272,1018072,6.880561,0,0.994249,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-01-05,6.880561,6.929418,6.853419,6.918832,2191283,6.744851,0,0.988689,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-01-06,6.744851,6.921276,6.731280,6.921276,1505531,6.812708,1,0.990040,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-01-07,6.812708,6.853421,6.750280,6.798594,1414040,6.975558,1,1.005005,1.0,0.984623,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-03-21,145.600006,146.949997,145.199997,145.699997,6482426,147.630005,1,0.998868,1.0,1.006693,2.0,0.944946,29.0,1.064466,142.0,1.407574,551.0
2025-03-24,147.630005,147.850006,146.289993,146.750000,1308316,148.630005,1,1.006923,1.0,1.016554,2.0,0.958675,29.0,1.078317,142.0,1.426189,552.0
2025-03-25,148.630005,149.770004,147.899994,148.449997,1599602,150.229996,1,1.003375,2.0,1.017233,3.0,0.965735,29.0,1.084604,142.0,1.434827,552.0


In [52]:
df_NAB

Price,Close,High,Low,Open,Volume,Tomorrow,Target,Close_Ratio_2,Trend_2,Close_Ratio_5,Trend_5,Close_Ratio_60,Trend_60,Close_Ratio_250,Trend_250,Close_Ratio_1000,Trend_1000
Date,,,,,,,,,,,,,,,,,
2000-01-03,5.492553,5.492553,5.492553,5.492553,0,5.303967,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-01-04,5.303967,5.421833,5.280394,5.419476,3005390,5.197887,0,0.982533,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-01-05,5.197887,5.228533,5.150741,5.183743,4604126,5.115383,0,0.989899,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-01-06,5.115383,5.256822,5.115383,5.255407,2574906,5.160172,1,0.992000,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-01-07,5.160172,5.174316,5.110668,5.138956,3028993,5.233247,1,1.004359,1.0,0.982143,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-03-21,33.180000,33.430000,32.990002,33.130001,11440658,33.910000,1,1.001358,2.0,1.006980,3.0,0.892467,29.0,0.915172,139.0,1.164688,550.0
2025-03-24,33.910000,33.910000,33.180000,33.230000,4515383,33.750000,0,1.010881,2.0,1.024967,4.0,0.913528,29.0,0.935226,139.0,1.189814,550.0
2025-03-25,33.750000,34.389999,33.709999,34.139999,4691013,33.980000,1,0.997635,1.0,1.013027,4.0,0.910784,28.0,0.930745,139.0,1.183709,550.0


In [53]:
df_CBA =df_CBA.dropna()

In [54]:
df_CBA

Price,Close,High,Low,Open,Volume,Tomorrow,Target,Close_Ratio_2,Trend_2,Close_Ratio_5,Trend_5,Close_Ratio_60,Trend_60,Close_Ratio_250,Trend_250,Close_Ratio_1000,Trend_1000
Date,,,,,,,,,,,,,,,,,
2003-11-03,9.137079,9.137079,9.061047,9.117245,2286983,9.226336,1,1.002539,1.0,1.003777,3.0,0.981058,27.0,1.031773,123.0,1.052296,484.0
2003-11-04,9.226336,9.252781,9.166832,9.209807,2441074,9.156910,0,1.004861,2.0,1.009112,4.0,0.991278,28.0,1.041458,124.0,1.062308,485.0
2003-11-05,9.156910,9.156910,9.084183,9.137075,3133740,9.104018,0,0.996223,1.0,1.000361,3.0,0.984574,28.0,1.033301,123.0,1.054038,485.0
2003-11-06,9.104018,9.140381,9.084184,9.110630,4720455,9.117240,1,0.997104,0.0,0.995733,2.0,0.979641,28.0,1.027071,122.0,1.047666,485.0
2003-11-07,9.117240,9.156909,9.104017,9.156909,2409170,9.057740,0,1.000726,1.0,0.996603,3.0,0.981774,29.0,1.028367,122.0,1.048909,485.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-03-20,145.929993,146.000000,143.679993,143.910004,3402826,145.600006,0,1.010946,1.0,1.013515,2.0,0.946377,30.0,1.067839,142.0,1.411731,552.0
2025-03-21,145.600006,146.949997,145.199997,145.699997,6482426,147.630005,1,0.998868,1.0,1.006693,2.0,0.944946,29.0,1.064466,142.0,1.407574,551.0
2025-03-24,147.630005,147.850006,146.289993,146.750000,1308316,148.630005,1,1.006923,1.0,1.016554,2.0,0.958675,29.0,1.078317,142.0,1.426189,552.0


In [55]:
df_NAB = df_NAB.dropna()

In [56]:
df_NAB

Price,Close,High,Low,Open,Volume,Tomorrow,Target,Close_Ratio_2,Trend_2,Close_Ratio_5,Trend_5,Close_Ratio_60,Trend_60,Close_Ratio_250,Trend_250,Close_Ratio_1000,Trend_1000
Date,,,,,,,,,,,,,,,,,
2003-11-03,8.351110,8.447733,8.337306,8.433930,3673077,8.389762,1,0.994412,0.0,0.985021,1.0,0.967799,23.0,0.963194,112.0,1.073855,495.0
2003-11-04,8.389762,8.436694,8.384240,8.403565,3143092,8.353870,0,1.002309,1.0,0.993008,2.0,0.973555,24.0,0.967780,113.0,1.078398,496.0
2003-11-05,8.353870,8.353870,8.260006,8.284853,4629539,8.237920,0,0.997856,1.0,0.993173,1.0,0.970728,24.0,0.963782,113.0,1.073349,496.0
2003-11-06,8.237920,8.315219,8.171663,8.315219,4710334,8.290371,1,0.993012,0.0,0.985925,1.0,0.958812,23.0,0.950646,112.0,1.058027,496.0
2003-11-07,8.290371,8.309696,8.218593,8.265525,2292645,8.204790,0,1.003173,1.0,0.995887,2.0,0.966428,23.0,0.956944,112.0,1.064335,496.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-03-20,33.090000,33.209999,32.709999,32.709999,8716437,33.180000,1,1.006693,2.0,1.003518,3.0,0.888469,29.0,0.912717,138.0,1.161991,550.0
2025-03-21,33.180000,33.430000,32.990002,33.130001,11440658,33.910000,1,1.001358,2.0,1.006980,3.0,0.892467,29.0,0.915172,139.0,1.164688,550.0
2025-03-24,33.910000,33.910000,33.180000,33.230000,4515383,33.750000,0,1.010881,2.0,1.024967,4.0,0.913528,29.0,0.935226,139.0,1.189814,550.0


**Improving The Model**

In [57]:
model1 = RandomForestClassifier(n_estimators = 200, min_samples_split=50, random_state=1)

In [58]:
model2 = RandomForestClassifier(n_estimators = 200, min_samples_split=50, random_state=1)

In [59]:
def predict1 (train1,test1,predictors_df1,model1):
  model1.fit(train1[predictors_df1],train1['Target'])
  preds1 = model1.predict_proba(test1[predictors_df1]) [:,1]
  preds1[preds1 >= .6] = 1
  preds1[preds1 < .6 ] = 0
  preds1 = pd.Series(preds1, index=test1.index, name='Predictions')
  combined1 = pd.concat([test1['Target'],preds1], axis=1)
  return combined1

In [60]:
def predict2 (train2,test2,predictors_df2,model2):
  model2.fit(train2[predictors_df2],train2['Target'])
  preds2 = model2.predict_proba(test2[predictors_df2]) [:,1]
  preds2[preds2 >= .6] = 1
  preds2[preds2 < .6 ] = 0
  preds2 = pd.Series(preds2, index=test2.index, name='Predictions')
  combined2 = pd.concat([test2['Target'],preds2], axis=1)
  return combined2

In [61]:
predictions1 = backtest1(df_CBA, model1, new_predictors1)

In [62]:
predictions2 = backtest2(df_NAB, model2, new_predictors2)

In [63]:
predictions1['Predictions'].value_counts()

,count
Predictions,
0.0,2760
1.0,161


In [64]:
predictions2['Predictions'].value_counts()

,count
Predictions,
0.0,2768
1.0,153


In [65]:
precision_score(predictions1['Target'], predictions1['Predictions'])

0.5838509316770186

In [66]:
precision_score(predictions2['Target'], predictions2['Predictions'])

0.5228758169934641

1. Evaluasi Model (report1 - Saham CBA.AX)
    *   Akurasi model: 0.48 (48%), yang menunjukkan model kurang baik dalam mengklasifikasikan data.
    *   Recall kelas 0 sangat tinggi (0.95), tapi kelas 1 sangat rendah (0.06) → Model cenderung hanya mengenali kelas 0, dan gagal mengenali kelas 1.
    * Precision kelas 1 lebih tinggi (0.58), tapi recall-nya rendah → Artinya model hanya sesekali benar saat memprediksi kelas 1, tetapi sering gagal menangkapnya.

2. Evaluasi Model (report2 - Saham 2)
    *   Akurasi model: 0.48 (48%), sama seperti model sebelumnya.
    *   Recall kelas 0 sangat tinggi (0.95), tapi kelas 1 sangat rendah (0.05) → Model ini juga hanya mengenali kelas 0 dan gagal mendeteksi kelas 1.
    * F1-score kelas 1 sangat rendah (0.10), yang berarti model hampir tidak bisa mengenali kelas tersebut.


In [67]:
report1 = classification_report(predictions1['Target'], predictions1['Predictions'])
print(report1)

              precision    recall  f1-score   support

           0       0.48      0.95      0.64      1386
           1       0.58      0.06      0.11      1535

    accuracy                           0.48      2921
   macro avg       0.53      0.51      0.37      2921
weighted avg       0.53      0.48      0.36      2921



In [68]:
report2 = classification_report(predictions2['Target'], predictions2['Predictions'])
print(report2)

              precision    recall  f1-score   support

           0       0.48      0.95      0.64      1401
           1       0.52      0.05      0.10      1520

    accuracy                           0.48      2921
   macro avg       0.50      0.50      0.37      2921
weighted avg       0.50      0.48      0.36      2921

